<h1> DattoRMM - Non-Compliant Software Scrape to CSV </h1>

In [ ]:
# general purpose
import time
import datetime as dt


# Spider
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
import re
import lxml

# Data Ingest and Export
import pandas as pd
import csv

In [ ]:
# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports/'

# import configparser for env secrets
from configparser import ConfigParser
config = ConfigParser()
config.read('d:/git/example_infrastructure_data_dev/config/env.ini')
import requests
from requests.structures import CaseInsensitiveDict

# Create Bot Object

## Create Firefox Driver

In [ ]:
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary

firefox_dev_binary = FirefoxBinary(r'C:/Program Files/Firefox Developer Edition/firefox.exe')
f_driver = webdriver.Firefox(firefox_binary=firefox_dev_binary, executable_path=r'D:/web_drivers/mozilla/geckodriver-v0.31.0-win64/geckodriver.exe')

## Create Chrome Driver

## Request Header Info (if needed to impersonate api browser calls)

In [ ]:
# construct header
headers = CaseInsensitiveDict()
headers['User-Agent'] = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'

# construct req body
data = CaseInsensitiveDict()
data = ''

In [ ]:
baseUrl = 'https://concord.centrastage.net'
auth_url = f'https://auth.datto.com'
#home_url = f'{baseUrl}'
links = []

# Login to DattoRMM

In [ ]:
f_driver.get(auth_url)

In [ ]:
title = f_driver.find_element_by_xpath('//h1')
title.text

In [ ]:
login_txtbox = f_driver.find_element_by_xpath("//input[@id='form_username']")
login_txtbox.send_keys((config['dattormm']['username']),Keys.ENTER)

In [ ]:
pass_txtbox = f_driver.find_element_by_xpath("//input[@id='form_password']")
pass_txtbox.send_keys((config['dattormm']['password']),Keys.ENTER)

In [ ]:
time.sleep(3)

In [ ]:
mfa_txtbox = f_driver.find_element_by_id('authy_token')
mfa_txtbox.send_keys('453731', Keys.ENTER) # number entered manually for now /TODO: automate MFA

# Non-Compliance Summary Page

In [ ]:
projectUrl = f'{baseUrl}/csm/system/softwaremanagement?sort=&order=&policyid=&c=All&selectItem=All&smstatus=NOT_COMPLIANT&query='
f_driver.get(projectUrl)

In [ ]:
tables = f_driver.find_elements_by_xpath('//table[@class="list"]')

In [ ]:
applications_table = tables[1].find_elements_by_xpath('.//tbody/tr/td')

In [ ]:
iter_col_labels = ['applicationName','verWin32','verWin64','vermacOS','nonComplientCount','blankCol']

pol_iter_counts = len(applications_table)/4

cols_label_iteration = []

for x in range(0,int(pol_iter_counts),1):
    for p in iter_col_labels:
        cols_label_iteration.append(p)

In [ ]:
app_summary_list = []
app_dicts = {}
targets_prog = re.compile(r'[^\,]+')
for label in range(0, len(applications_table)):
    app_dict = {}
    #print(cols_label_iteration[label])
    #print(applications_table[label].text)
    try:

        if cols_label_iteration[label] == 'blankCol':
            app_summary_list.append(app_dicts)
            app_dicts = {}
        elif (cols_label_iteration[label] == 'nonComplientCount') & (applications_table[label].text != 0):
            app_dict[cols_label_iteration[label]] = applications_table[label].text
            h = applications_table[label].find_elements_by_xpath('./a')
            for x in h:
                app_dict['href'] = x.get_attribute('href')

            app_dicts.update(app_dict)
        else:
            #print('-'*100)
            app_dict[cols_label_iteration[label]] = applications_table[label].text
            app_dicts.update(app_dict)
            #app_summary_list.append(app_dict)
    except Exception as e:
        print(e)

In [ ]:
df = pd.DataFrame(app_summary_list)

In [ ]:
df

In [ ]:
df.to_csv('.csv',index=False)

In [ ]:
href_list[0]

In [ ]:
f_driver.get(href_list[0])

In [ ]:
tr_rows = f_driver.find_elements_by_xpath('//tr[contains(@class,"devicerow")]')

In [ ]:
app_device_list = []
for tr in tr_rows:
    td = tr.find_elements_by_xpath('.//td')
    for t in td:
        t_text = t.text
        if t_text == '':
            continue
        else:
            app_device_list.append(t_text)
    #print(t.get_attribute('href').text())

In [ ]:
app_device_list

In [ ]:
iter_col_labels = ['uid','sitename','hostname','lastSeen']
table_label_iteration = []
pol_iter_counts = len(app_device_list)/len(iter_col_labels)


for x in range(0,int(pol_iter_counts)+1):
    for p in iter_col_labels:
        table_label_iteration.append(p)

In [ ]:
non_compliant_device_list = []
device_dicts = {}
targets_prog = re.compile(r'[^\,]+')
for label in range(0, len(app_device_list)):
    device_dict = {}
    #print(cols_label_iteration[label])
    #print(applications_table[label].text)
    try:

        if cols_label_iteration[label] == 'blankCol':
            non_compliant_device_list.append(device_dicts)
            app_dicts = {}

        else:
            #print('-'*100)
            device_dict[table_label_iteration[label]] = app_device_list[label]
            device_dicts.update(device_dict)
            #app_summary_list.append(app_dict)
    except Exception as e:
        print(e)

In [ ]:
non_compliant_device_list

In [ ]:
df = pd.DataFrame(non_compliant_device_list)

In [ ]:
df

In [ ]:
df = pd.read_clipboard()

In [ ]:
df